In [1]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from collections import defaultdict
import faiss

In [2]:
# Load in uniprot meta data
meta_data_new = pd.read_csv('/home/ron/protein-vec/src_run/data/uniprotkb_AND_reviewed_true_2023_07_03.tsv', sep='\t')

In [3]:
meta_data_new.shape

(569793, 16)

In [4]:
#Now filter for the proteins that were newly discovered
new_proteins = meta_data_new[meta_data_new['Date of creation'] > '2022-05-25'].reset_index(drop=True)

# drop the new proteins and save to lookup
lookup_proteins = meta_data_new[~meta_data_new['Entry'].isin(new_proteins['Entry'])].reset_index(drop=True)

In [5]:
new_proteins.shape

(2350, 16)

In [6]:
lookup_proteins_meta = pd.read_csv('/home/seyonec/protein-vec/src_run/protein_vec_embeddings/lookup_embeddings_meta_data.tsv', sep="\t")
# were all of these trained on?

In [7]:
lookup_proteins_meta.shape

(540560, 17)

In [8]:
# new protein Entry is in the lookup database Entry
new_proteins['Entry'].isin(lookup_proteins_meta['Entry']).sum()


0

In [9]:
# get proteins in meta_data_new that are not in lookup_proteins_meta
new_proteins_not_in_lookup = new_proteins[~new_proteins['Entry'].isin(lookup_proteins_meta['Entry'])].reset_index(drop=True)

In [10]:
len(new_proteins_not_in_lookup), len(new_proteins)
# exact match, so no leakage

(2350, 2350)

In [11]:
lookup_embeddings = np.load('/home/seyonec/protein-vec/src_run/protein_vec_embeddings/lookup_embeddings.npy')

In [12]:
lookup_embeddings.shape
# we should randomly sample some subset of 15k proteins for lookup? or just use all of them?

(540560, 512)

In [13]:
column = 'EC number'

In [14]:
lookup_proteins_meta

,Unnamed: 0,Entry,Reviewed,Entry Name,Protein names,Gene Names,Organism,Length,Gene Ontology (biological process),Gene Ontology (cellular component),Gene Ontology (molecular function),Gene Ontology (GO),EC number,Sequence,Date of creation,Gene3D,Pfam
0,0,A0A009IHW8,reviewed,ABTIR_ACIB9,2' cyclic ADP-D-ribose synthase AbTIR (2'cADPR...,J512_3302,Acinetobacter baumannii (strain 1295743),269,NAD catabolic process [GO:0019677]; signal tra...,NaN,NAD(P)+ nucleosidase activity [GO:0050135]; NA...,NAD(P)+ nucleosidase activity [GO:0050135]; NA...,3.2.2.-; 3.2.2.6,MSLEQKKGADIISKILQIQNSIGKTTSPSTLKTKLSEISRKEQENA...,2020-02-26,3.40.50.10140;,PF13676;
1,1,A0A011QK89,reviewed,L2HDH_ACCRE,L-2-hydroxyglutarate dehydrogenase (EC 1.1.99.2),lhgO_1 AW11_01282,Accumulibacter regalis,388,NaN,NaN,(S)-2-hydroxy-acid oxidase activity [GO:000397...,(S)-2-hydroxy-acid oxidase activity [GO:000397...,1.1.99.2,MESIEAVVIGAGVVGLACARELARRGFETVILERHGAFGTETSARN...,2022-02-23,3.30.9.10;3.50.50.60;,PF01266;
2,2,A0A023FBW4,reviewed,E1142_AMBCJ,Evasin P1142,NaN,Amblyomma cajennense (Cayenne tick) (Acarus ca...,97,NaN,extracellular region [GO:0005576],C-X-C chemokine binding [GO:0019958],extracellular region [GO:0005576]; C-X-C chemo...,NaN,MTSHGAVKIAIFAVIALHSIFECLSKPQILQRTDHSTDSDWDPQMC...,2020-12-02,NaN,NaN
3,3,A0A023FBW7,reviewed,EV546_AMBCJ,Evasin P546,NaN,Amblyomma cajennense (Cayenne tick) (Acarus ca...,118,negative regulation of chemokine activity [GO:...,extracellular region [GO:0005576],C-C chemokine binding [GO:0019957],extracellular region [GO:0005576]; C-C chemoki...,NaN,MKVLLYIAASCLMLLALNVSAENTQQEEEDYDYGTDTCPFPVLANK...,2020-12-02,2.30.130.100;,PF19429;
4,4,A0A023FDY8,reviewed,EV974_AMBCJ,Evasin P974,NaN,Amblyomma cajennense (Cayenne tick) (Acarus ca...,118,negative regulation of chemokine activity [GO:...,extracellular region [GO:0005576],C-C chemokine binding [GO:0019957],extracellular region [GO:0005576]; C-C chemoki...,NaN,MKVLLCIAASCLMLLALNVSAENTQQEEQDYDYGTDTCPFPVLANK...,2020-12-02,2.30.130.100;,PF19429;
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
540555,540555,X5IWT5,reviewed,CSA8A_CONGE,Alpha-conotoxin GVIIIB (AlphaS-conotoxin GVIIIB),NaN,Conus geographus (Geography cone) (Nubecula ge...,88,NaN,extracellular region [GO:0005576]; host cell p...,acetylcholine receptor inhibitor activity [GO:...,extracellular region [GO:0005576]; host cell p...,NaN,MMSKMGAMFVLLLLFTLASSQQEGDVQARKTRPKSDFYRALPRSGS...,2018-05-23,NaN,NaN
540556,540556,X5IY26,reviewed,I3B1_CONGE,Conotoxin G11.1 (Conotoxin G117) (Conotoxin GXIA),NaN,Conus geographus (Geography cone) (Nubecula ge...,77,NaN,extracellular region [GO:0005576],ion channel regulator activity [GO:0099106]; t...,extracellular region [GO:0005576]; ion channel...,NaN,MKLFLAIVLILMLQFLSTGAETSDNHASRSTTALRDWLLGPKAKRC...,2019-10-16,NaN,NaN
540557,540557,X5JA13,reviewed,SECAA_ARATH,Exocyst complex component SEC10a (AtSec10a) (E...,SEC10a At5g12370 T2L20.2,Arabidopsis thaliana (Mouse-ear cress),825,exocytosis [GO:0006887]; Golgi to plasma membr...,cytosol [GO:0005829]; exocyst [GO:0000145]; ex...,NaN,cytosol [GO:0005829]; exocyst [GO:0000145]; ex...,NaN,MTEGIRARGPRSSSVNSVPLILDIEDFKGDFSFDALFGNLVNDLLP...,2017-01-18,1.20.58.1970;,PF07393;
540558,540558,X5JB51,reviewed,SECAB_ARATH,Exocyst complex component SEC10b (AtSec10b),SEC10b At5g12365,Arabidopsis thaliana (Mouse-ear cress),829,acceptance of pollen [GO:0060321]; exocytosis ...,cytosol [GO:0005829]; exocyst [GO:0000145]; ex...,NaN,cytosol [GO:0005829]; exocyst [GO:0000145]; ex...,NaN,MTERIRARGPRSSSVNSVPLILDIEDFKGDFSFDALFGNLVNDLLP...,2017-01-18,1.20.58.1970;,PF07393;


In [15]:
# select proteins and their embeddings only for proteins that have a EC number (fully characterized)
col_proteins_meta = lookup_proteins_meta[~lookup_proteins_meta[column].isnull()]
col_proteins_meta = col_proteins_meta[~col_proteins_meta[column].str.split('.').apply(lambda x: len(x) > 4)]
col_proteins_meta = col_proteins_meta[~col_proteins_meta[column].str.split('.').apply(lambda x: x[-1] == '-')]

col_embeddings = lookup_embeddings[col_proteins_meta.index]
# drop index to avoid confusion
col_proteins_meta = col_proteins_meta.reset_index(drop=True)

In [16]:
col_proteins_meta.shape, col_embeddings.shape

((211720, 17), (211720, 512))

In [17]:
col_proteins_meta

,Unnamed: 0,Entry,Reviewed,Entry Name,Protein names,Gene Names,Organism,Length,Gene Ontology (biological process),Gene Ontology (cellular component),Gene Ontology (molecular function),Gene Ontology (GO),EC number,Sequence,Date of creation,Gene3D,Pfam
0,1,A0A011QK89,reviewed,L2HDH_ACCRE,L-2-hydroxyglutarate dehydrogenase (EC 1.1.99.2),lhgO_1 AW11_01282,Accumulibacter regalis,388,NaN,NaN,(S)-2-hydroxy-acid oxidase activity [GO:000397...,(S)-2-hydroxy-acid oxidase activity [GO:000397...,1.1.99.2,MESIEAVVIGAGVVGLACARELARRGFETVILERHGAFGTETSARN...,2022-02-23,3.30.9.10;3.50.50.60;,PF01266;
1,107,A0A023W421,reviewed,DTD_BACAM,D-aminoacyl-tRNA deacylase (DTD) (EC 3.1.1.96)...,dtd,Bacillus amyloliquefaciens (Bacillus velezensis),147,D-amino acid catabolic process [GO:0019478],cytoplasm [GO:0005737],D-aminoacyl-tRNA deacylase activity [GO:005149...,cytoplasm [GO:0005737]; D-aminoacyl-tRNA deacy...,3.1.1.96,MKLVVQRVTDASVTVDGAVAGRIGPGIMALVGVTHEDTEEDAAYLA...,2017-09-27,3.50.80.10;,PF02580;
2,115,A0A024BTN9,reviewed,OXLA_BOTSC,L-amino acid oxidase Bs29 (BsLAAO) (LAO) (EC 1...,NaN,Bothriechis schlegelii (Eyelash palm pitviper),498,defense response to bacterium [GO:0042742],extracellular region [GO:0005576],L-amino-acid oxidase activity [GO:0001716],extracellular region [GO:0005576]; L-amino-aci...,1.4.3.2,SCADDRNPLEECFQETDYEEFLEIARNGLKATSNPKHVVIVGAGMS...,2014-10-29,3.90.660.10;3.50.50.60;1.10.405.10;,PF01593;
3,120,A0A024RBG1,reviewed,NUD4B_HUMAN,Diphosphoinositol polyphosphate phosphohydrola...,NUDT4B,Homo sapiens (Human),181,adenosine 5'-(hexahydrogen pentaphosphate) cat...,cytoplasm [GO:0005737]; cytosol [GO:0005829]; ...,bis(5'-adenosyl)-hexaphosphatase activity [GO:...,cytoplasm [GO:0005737]; cytosol [GO:0005829]; ...,3.6.1.52,MMKFKPNQTRTYDREGFKKRAACLCFRSEQEDEVLLVSSSRYPDQW...,2018-07-18,3.90.79.10;,PF00293;
4,121,A0A024RXP8,reviewed,GUX1_HYPJR,"Exoglucanase 1 (EC 3.2.1.91) (1,4-beta-cellobi...",cbh1 M419DRAFT_125125,Hypocrea jecorina (strain ATCC 56765 / BCRC 32...,514,cellulose catabolic process [GO:0030245],extracellular region [GO:0005576],"cellulose 1,4-beta-cellobiosidase activity [GO...",extracellular region [GO:0005576]; cellulose 1...,3.2.1.91,MYRKLAVISAFLATARAQSACTLQSETHPPLTWQKCSSGGTCTQQT...,2017-08-30,2.70.100.10;,PF00734;PF00840;
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211715,540543,X2JI34,reviewed,GAO_TANPA,Germacrene A hydroxylase (EC 1.14.14.95) (Germ...,NaN,Tanacetum parthenium (Feverfew) (Matricaria pa...,488,sesquiterpene biosynthetic process [GO:0051762...,endoplasmic reticulum membrane [GO:0005789],germacrene A hydroxylase activity [GO:0106223]...,endoplasmic reticulum membrane [GO:0005789]; g...,1.14.14.95,MALSLTTSIALATILFFVYKFATRSKSTKNSLPEPWRLPIIGHMHH...,2019-10-16,1.10.630.10;,PF00067;
211716,540544,X2L4E2,reviewed,OXLA_BOTPC,L-amino acid oxidase (Bpic-LAAO) (LAO) (EC 1.4...,NaN,Bothrops pictus (Desert lancehead),498,apoptotic process [GO:0006915]; defense respon...,extracellular region [GO:0005576],L-amino-acid oxidase activity [GO:0001716]; to...,extracellular region [GO:0005576]; L-amino-aci...,1.4.3.2,SLLFLAAVGSCADDRNPLEECFRETDYEEFLEIAKNGLSTTSNPKR...,2015-04-29,3.90.660.10;3.50.50.60;1.10.405.10;,PF01593;
211717,540545,X4Y2L4,reviewed,LHYAL_HIRNI,Hyaluronoglucuronidase (LHyal) (EC 3.2.1.36) (...,NaN,Hirudo nipponia (Korean blood-sucking leech),489,hyaluronan catabolic process [GO:0030214],membrane [GO:0016020],hyaluronoglucuronidase activity [GO:0033906]; ...,membrane [GO:0016020]; hyaluronoglucuronidase ...,3.2.1.36,MKEIAVTIDDKNVIASVSESFHGVAFDASLFSPKGLWSFVDITSPK...,2017-06-07,3.20.20.80;,PF03662;
211718,540548,X5CY81,reviewed,CNDC1_RHIWD,"Chloroacetanilide N-alkylformylase, ferredoxin...",cndC1 HY78_15240,Rhizorhabdus wittichii (strain DC-6 / KACC 166...,410,NaN,NaN,ferredoxin-NAD+ reductase activity [GO:0008860...,ferredoxin-NAD+ reductase activity [GO:0008860...,1.18.1.3,MAQYDVLIVGAGHGGAQAAVALRQNKFEGTIAIVGDEPELPYERPP...,2018-10-10,3.30.390.30;3.50.50.60;,PF07992;PF14759;


### New proteins (test) embed and preprocess

In [18]:
new_protein_embeddings = np.load('/home/seyonec/protein-vec/src_run/protein_vec_embeddings/new_protein_embeddings.npy')
new_protein_embeddings.shape

(2295, 512)

In [19]:
print('Number of new proteins deposited after 2022-05-25')
print(len(new_proteins))

# filter those proteins that are greater than 2000 amino acid residues long
new_proteins['length'] = new_proteins['Sequence'].str.len()
new_proteins = new_proteins[new_proteins['length'] <= 2000]
print('Filtered proteins longer than 2000 amino acids')
print(len(new_proteins))
new_proteins = new_proteins.reset_index(drop=True)

Number of new proteins deposited after 2022-05-25
2350
Filtered proteins longer than 2000 amino acids
2295


In [30]:
new_proteins

,Entry,Reviewed,Entry Name,Protein names,Gene Names,Organism,Length,Gene Ontology (biological process),Gene Ontology (cellular component),Gene Ontology (molecular function),Gene Ontology (GO),EC number,Sequence,Date of creation,Gene3D,Pfam,length
0,A0A023I7E1,reviewed,ENG1_RHIMI,"Glucan endo-1,3-beta-D-glucosidase 1 (Endo-1,3...",ENG1 LAM81A,Rhizomucor miehei,796,cell wall organization [GO:0071555]; polysacch...,extracellular region [GO:0005576],"glucan endo-1,3-beta-D-glucosidase activity [G...",extracellular region [GO:0005576]; glucan endo...,3.2.1.39,MRFQVIVAAATITMITSYIPGVASQSTSDGDDLFVPVSNFDPKSIF...,2022-12-14,1.10.287.1170;1.20.5.420;,PF17652;PF03639;,796
1,A0A061AE05,reviewed,PAPSH_CAEEL,Bifunctional 3'-phosphoadenosine 5'-phosphosul...,pps-1 T14G10.1,Caenorhabditis elegans,654,3'-phosphoadenosine 5'-phosphosulfate biosynth...,nucleus [GO:0005634],adenylylsulfate kinase activity [GO:0004020]; ...,nucleus [GO:0005634]; adenylylsulfate kinase a...,2.7.1.25; 2.7.7.4,MLTPRDENNEGDAMPMLKKPRYSSLSGQSTNITYQEHTISREERAA...,2023-02-22,3.40.50.620;3.40.50.300;3.10.400.10;,PF01583;PF01747;PF14306;,654
2,A0A072VDF2,reviewed,CCR1_MEDTR,Cinnamoyl-CoA reductase 1 (Mt-CCR1) (EC 1.2.1....,CCR1 MTR_2g104960 MtrunA17_Chr2g0333781,Medicago truncatula (Barrel medic) (Medicago t...,342,lignin biosynthetic process [GO:0009809]; phen...,cytoplasm [GO:0005737],cinnamoyl-CoA reductase activity [GO:0016621];...,cytoplasm [GO:0005737]; cinnamoyl-CoA reductas...,1.2.1.-; 1.2.1.44,MPAATAAAAAESSSVSGETICVTGAGGFIASWMVKLLLEKGYTVRG...,2023-02-22,3.40.50.720;,PF01370;,342
3,A0A076FFM5,reviewed,F8H1_OCIBA,"Flavonoid 8-hydroxylase 1, chloroplastic (ObF8...",F8H-1,Ocimum basilicum (Sweet basil),523,flavonoid metabolic process [GO:0009812],chloroplast [GO:0009507]; chloroplast membrane...,"2 iron, 2 sulfur cluster binding [GO:0051537];...",chloroplast [GO:0009507]; chloroplast membrane...,1.14.15.-,MPFPMEVLQASSLSFPLLRRHSRNNLINKFRNPTLPRIDIPRQNID...,2022-12-14,2.102.10.10;,PF08417;PF00355;,523
4,A0A0B4U9L8,reviewed,VMF1_VIPAA,Zinc metalloproteinase-disintegrin-like protei...,NaN,Vipera ammodytes ammodytes (Western sand viper),614,envenomation resulting in modulation of blood ...,extracellular region [GO:0005576]; extraorgani...,metalloendopeptidase activity [GO:0004222]; me...,extracellular region [GO:0005576]; extraorgani...,3.4.24.-,MLQVLLVTICLAVFPYQGSSIILESGNVNDYEVVYPQKLTALLKGA...,2023-02-22,3.40.390.10;4.10.70.10;,PF08516;PF00200;PF01562;PF01421;,614
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2290,P71839,reviewed,Y786_MYCTU,Protein Rv0786c,Rv0786c,Mycobacterium tuberculosis (strain ATCC 25618 ...,212,NaN,NaN,NaN,NaN,NaN,MQLTHFGHSCLLAEFGQTRLLFDPGTFSHGFEGITGLSAILITHQH...,2022-08-03,3.60.15.10;,PF13483;,212
2291,P9WEN8,reviewed,YH52B_YEAST,Uncharacterized protein YHR052C-B,YHR052C-B,Saccharomyces cerevisiae (strain ATCC 204508 /...,57,NaN,NaN,NaN,NaN,NaN,MKVMSANANVVAAKIMNNAKNHVAAQRGVTATTNAPAVTSLKKPRS...,2022-08-03,NaN,NaN,57
2292,P9WEN9,reviewed,YH54B_YEAST,Uncharacterized protein YHR054C-B,YHR054C-B,Saccharomyces cerevisiae (strain ATCC 204508 /...,57,NaN,NaN,NaN,NaN,NaN,MKVMSANANVVAAKIMNNAKNHVAAQRGVTATTNAPAVTSLKKPRS...,2022-08-03,NaN,NaN,57
2293,P9WES8,reviewed,CT930_CHATD,Uncharacterized protein CTHT_0020930_1,CTHT_0020930_1,Chaetomium thermophilum (strain DSM 1495 / CBS...,135,2-oxoglutarate metabolic process [GO:0006103],mitochondrial oxoglutarate dehydrogenase compl...,NaN,mitochondrial oxoglutarate dehydrogenase compl...,NaN,MFGTKALRAAQHAVRTPSIKFIGKRTIPAVVDHTPQPHPASPTKTL...,2022-10-12,NaN,PF10937;,135


In [20]:
# select proteins and their embeddings only for proteins that have a EC number (fully characterized)
column = 'EC number'
col_new_proteins_meta = new_proteins[~new_proteins[column].isnull()]
col_new_proteins_meta = col_new_proteins_meta[~col_new_proteins_meta[column].str.split('.').apply(lambda x: len(x) > 4)]
col_new_proteins_meta = col_new_proteins_meta[~col_new_proteins_meta[column].str.split('.').apply(lambda x: x[-1] == '-')]

col_new_embeddings = new_protein_embeddings[col_new_proteins_meta.index]
# drop index to avoid confusion
col_new_proteins_meta = col_new_proteins_meta.reset_index(drop=True)

In [21]:
col_new_proteins_meta.shape, col_new_embeddings.shape

((438, 17), (438, 512))

In [22]:
## Filter the lookup proteins to have only 'EC Number' that are in col_new_proteins_meta['EC number']
col_proteins_meta_in_lookup = col_proteins_meta[col_proteins_meta['EC number'].isin(col_new_proteins_meta['EC number'])]
col_embeddings = col_embeddings[col_proteins_meta_in_lookup.index]
col_proteins_meta_in_lookup = col_proteins_meta_in_lookup.reset_index(drop=True)


In [23]:
col_proteins_meta_in_lookup.shape, col_embeddings.shape

((29294, 17), (29294, 512))

In [25]:
from protein_conformal.util import load_database, query

# load database
lookup_database = load_database(col_embeddings)
k = len(col_proteins_meta_in_lookup)
# Query for the 1st nearest neighbor
D, I = query(lookup_database, col_new_embeddings, k)

In [26]:
D.shape

(438, 29294)

In [27]:
I.shape

(438, 29294)

In [31]:
# save D, I, col_query, col_lookup
np.save('lookup_v_test_date_cutoff_faiss_query_results.npy', D)
np.save('lookup_v_test_date_cutoff_faiss_query_indices.npy', I)
col_proteins_meta_in_lookup.to_csv('lookup_embeddings_faiss_query_meta_data.tsv', sep="\t")
col_new_proteins_meta.to_csv('test_embeddings_faiss_lookup_meta_data.tsv', sep="\t")


In [33]:
col_new_proteins_meta

,Entry,Reviewed,Entry Name,Protein names,Gene Names,Organism,Length,Gene Ontology (biological process),Gene Ontology (cellular component),Gene Ontology (molecular function),Gene Ontology (GO),EC number,Sequence,Date of creation,Gene3D,Pfam,length
0,A0A023I7E1,reviewed,ENG1_RHIMI,"Glucan endo-1,3-beta-D-glucosidase 1 (Endo-1,3...",ENG1 LAM81A,Rhizomucor miehei,796,cell wall organization [GO:0071555]; polysacch...,extracellular region [GO:0005576],"glucan endo-1,3-beta-D-glucosidase activity [G...",extracellular region [GO:0005576]; glucan endo...,3.2.1.39,MRFQVIVAAATITMITSYIPGVASQSTSDGDDLFVPVSNFDPKSIF...,2022-12-14,1.10.287.1170;1.20.5.420;,PF17652;PF03639;,796
1,A0A0D2Y5A7,reviewed,ODP2_FUSO4,Dihydrolipoyllysine-residue acetyltransferase ...,LAT1 FOXG_11462,Fusarium oxysporum f. sp. lycopersici (strain ...,457,acetyl-CoA biosynthetic process from pyruvate ...,mitochondrial pyruvate dehydrogenase complex [...,dihydrolipoyllysine-residue acetyltransferase ...,mitochondrial pyruvate dehydrogenase complex [...,2.3.1.12,MLSAALRRRVLAPTHSALRTGFAAHVVRHYASFPEHQVIKMPALSP...,2023-05-03,2.40.50.100;3.30.559.10;4.10.320.10;,PF00198;PF00364;PF02817;,457
2,A0A0H2Z7X0,reviewed,TPBB_PSEAB,Diguanylate cyclase TpbB (EC 2.7.7.65),tpbB yfiN PA14_49890,Pseudomonas aeruginosa (strain UCBPP-PA14),435,signal transduction [GO:0007165],membrane [GO:0016020],NaN,membrane [GO:0016020]; signal transduction [GO...,2.7.7.65,MNRRRRYTGSNPSLRRVLYRAHLGVALVAVFTAGLAVTLVGLLTLR...,2023-06-28,3.30.70.270;6.10.340.10;,PF17152;PF00990;PF00672;,435
3,A0A0K2JL82,reviewed,CRED_STRCM,Nitrosuccinate lyase (EC 4.3.99.5),creD,Streptomyces cremeus,476,"3,4-dihydroxybenzoate catabolic process [GO:00...",NaN,"3-carboxy-cis,cis-muconate cycloisomerase acti...","3-carboxy-cis,cis-muconate cycloisomerase acti...",4.3.99.5,MTRPPAPPPGAPGADELLDCGLLSPVRAGTPVEALVCDSAWLQAML...,2023-02-22,1.10.40.30;1.20.200.10;1.10.275.10;,PF10397;PF00206;,476
4,A0A144A134,reviewed,ISN1_PLAF7,IMP-specific 5'-nucleotidase 1 (PfISN1) (EC 3....,ISN1 PF3D7_1206100,Plasmodium falciparum (isolate 3D7),444,IMP catabolic process [GO:0006204]; inosine sa...,cytoplasm [GO:0005737],ATP binding [GO:0005524]; IMP 5'-nucleotidase ...,cytoplasm [GO:0005737]; ATP binding [GO:000552...,3.1.3.99,MKNLDINTFDNIEDIPLGSSEQDPYDFFTLSDRNVMNSDMKKNIVQ...,2022-12-14,3.40.50.1000;,PF06437;,444
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
433,P95598,reviewed,ROX_RHOHA,Rifampicin monooxygenase (RIFMO) (EC 1.14.13.211),iri,Rhodococcus hoagii (Corynebacterium equii),479,NaN,NaN,FAD binding [GO:0071949]; monooxygenase activi...,FAD binding [GO:0071949]; monooxygenase activi...,1.14.13.211,MSDVIIVGAGPTGLMLAGELRLQGVDVVVVDKDEEPTQFVRALGIH...,2022-12-14,3.30.70.2450;3.40.30.120;3.50.50.60;,PF01494;,479
434,P96274,reviewed,HAT2_MYCTU,Probable histone acetyltransferase Rv0428c (EC...,Rv0428c,Mycobacterium tuberculosis (strain ATCC 25618 ...,302,NaN,NaN,acyltransferase activity [GO:0016746],acyltransferase activity [GO:0016746],2.3.1.48,MVSWPGLGTRVTVRYRRPAGSMPPLTDAVGRLLAVDPTVRVQTKTG...,2022-10-12,NaN,NaN,302
435,P9WES7,reviewed,CYT2_CHATD,Putative holocytochrome-c1 synthase (EC 4.4.1....,CTHT_0020930_2,Chaetomium thermophilum (strain DSM 1495 / CBS...,287,NaN,mitochondrial inner membrane [GO:0005743],holocytochrome-c synthase activity [GO:0004408...,mitochondrial inner membrane [GO:0005743]; hol...,4.4.1.17,MRGFGSDSSQASGEEAKCPVNHKTRELWLHQARAAQSASTSAAPAL...,2022-10-12,NaN,PF01265;,287
436,Q5LT52,reviewed,TDM_RUEPO,Trimethylamine-oxide aldolase (EC 4.1.2.32) (T...,tdm SPO1562,Ruegeria pomeroyi (strain ATCC 700808 / DSM 15...,789,NaN,NaN,NaN,NaN,4.1.2.32,MLDTKYPEIIPGPPKPSAILQPRVFSLPPGTERYVVPGAGAILLRL...,2023-06-28,NaN,PF09347;PF01571;PF08669;,789


In [37]:
from protein_conformal.util import scope_hierarchical_loss
def get_scope_dict(true_test_idcs, test_df, lookup_idcs, lookup_df, D, I, id = 'Entry', label = 'EC number'):
    """
    true_test_idcs: indices of the test set within the scope dataframe

    test_df: dataframe containing the test set (indices are the same as true_test_idcs)

    lookup_idcs: indices of the lookup set within the larger scope dataframe

    lookup_df: dataframe containing the lookup set (indices are the same as lookup_idcs)

    D: distances matrix (400 x 14777 or test x lookup by default)

    I: indices matrix (400 x 14777 or test x lookup by default)

    NOTE: Indices computed by FAISS are not the same as the indices of the dataframe, so 
    we use the lookup_idcs list to map FAISS indices in I to the indices of the dataframe
    """

    near_ids = []
    min_sim = np.min(D)
    max_sim = np.max(D)

    for i in range(len(true_test_idcs)):
        test_id = test_df.loc[true_test_idcs[i], id]
        test_sccs = test_df.loc[true_test_idcs[i], label]
        query_ids = [lookup_df.loc[lookup_idcs[j], id] for j in I[i]]
        exact_loss = [scope_hierarchical_loss(test_sccs, lookup_df.loc[lookup_idcs[j], label]) for j in I[i]]
        # grab the 2nd element in the tuple belonging to each element of exact_loss as mask_exact
        mask_exact = [x[1] for x in exact_loss]
        loss = [x[0] for x in exact_loss]
        
        # define mask_partial as 1 for any element of loss that is <=1 (tolerate retrieving homolog with diff family but same superfamily)
        mask_partial = [l <= 1 for l in loss]

        # create a row of size len(lookup_df) where each element is the sum of all entries in S_i until that index
        sum = np.cumsum(D[i])
        norm_sim = (D[i] - min_sim) / (max_sim - min_sim) # convert similarities into a probability space (0, 1) based on (min_sim, max_sim)
        #mask_exact = [test_sccs == lookup_df.loc[lookup_idcs[j], 'sccs'] for j in I[i]]

        sum_norm_s_i = np.cumsum(norm_sim)
        near_ids.append({
            'test_id': test_id,
            'query_ids': query_ids,
            #'meta_query': meta_query,
            'loss' : loss,
            'exact': mask_exact,
            'partial': mask_partial,
            'S_i': D[i],
            'Sum_i' : sum,
            'Norm_S_i' : norm_sim,
            'Sum_Norm_S_i': sum_norm_s_i,
            'I_i': I[i]
        })
    return near_ids


In [38]:
uniprot_ec_dict = get_scope_dict(list(range(len(col_new_proteins_meta))), col_new_proteins_meta, list(range(len(col_proteins_meta_in_lookup))), col_proteins_meta_in_lookup, D, I)
# before, we used the raw indices from the original scope dataframe as the indices for the test and lookup dataframes, but here we adjust to have their own indices

In [42]:
uniprot_ec_dict[300]['loss']

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 3,
 4,
 4,
 4,
 3,
 3,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 1,
 1,
 1,
 4,
 4,
 3,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 3,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 1,
 3,
 4,
 3,
 4,
 4,
 4,
 3,
 4,
 4,
 3,
 3,
 3,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 3,
 3,
 3,
 4,
 4,
 4,
 4,
 4,
 4,
 3,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 3,
 3,
 3,
 3,
 4,
 3,
 4,
 4,
 4,
 4,
 3,
 4,
 3,
 3,
 4,
 4,
 4,
 3,
 3,
 4,
 4,
 4,
 4,
 4,
 4,
 3,
 4,
 3,
 3,
 3,
 4,
 4,
 4,
 4,
 4,
 3,
 3,
 3,
 4,
 4,
 4,
 4,
 3,
 4,
 4,
 3,
 3,
 3,
 4,
 4,
 3,
 3,
 4,
 3,
 4,
 4,
 4,
 4,
 3,
 3,
 4,
 4,
 4,
 3,
 4,
 3,
 4,
 3,
 4,
 4,
 4,
 4,
 3,
 4,
 3,
 4,
 3,
 4,
 4,
 3,
 4,
 4,
 4,
 4,
 3,
 4,
 4,
 3,
 4,
 4,
 4,
 4,
 4,
 4,


In [43]:
np.save('uniprot_ec_test_v_lookup.npy', uniprot_ec_dict)
